In [3]:
from openai import OpenAI
import random
import json
from datetime import datetime

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# ------------------------------------------------------------
# ЧАСТЬ 1. СБОР ЖАЛОБ (БЕЗ ИИ)
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ЗДРАВСТВУЙТЕ! Я ЛЮДМИЛА, МЕДИЦИНСКИЙ АССИСТЕНТ")
print("=" * 60)
print("\nЯ помогу собрать информацию о вашем самочувствии")
print("перед беседой с врачом. Всё, что вы расскажете,")
print("я передам доктору.")
print("-" * 60)

# Знакомство
name = input("\n👤 Как вас зовут? ")
print(f"\n🤖 Людмила: Очень приятно, {name}!")

# Сбор жалоб
print("\n🤖 Людмила: Расскажите, пожалуйста о том, что Вас беспокоит?")

complaints = []
questions = [
    "Что-нибудь ещё?",
    "Может что-то ещё?",
    "Что ещё вас беспокоит?",
    "Я Вас внимательно слушаю, продолжайте, что ещё?"
]

while True:
    complaint = input("👤 Вы: ").strip()
    
    # Проверка на завершение
    if any(phrase in complaint.lower() for phrase in 
           ["это всё", "нет", "пожалуй нет", "это основное", "больше нет", "наверное всё", "всё", "хватит"]):
        break
    
    if complaint:  # если не пустой ответ
        complaints.append(complaint)
        
        # Разные варианты следующих вопросов
        next_q = random.choice(questions)
        print(f"\n🤖 Людмила: {next_q}")

# Подведение итога
if complaints:    
    points = "\n".join([f"   • {жалоба}" for жалоба in complaints])
    summary = f"Правильно ли я поняла, что вас беспокоит:\n{points}"  
else:
    summary = "Правильно ли я поняла, что сейчас вас ничего не беспокоит?"

print(f"\n🤖 Людмила: {summary}")
confirmation = input("👤 Вы: ")

# Корректировка при необходимости
corrections = []
if "не" in confirmation.lower() or "нет" in confirmation.lower():
    print("\n🤖 Людмила: Поправьте меня, пожалуйста. Что именно я поняла неверно?")
    correction = input("👤 Вы: ")
    corrections.append(correction)
    # Здесь можно добавить логику уточнения, но для MVP достаточно

# ------------------------------------------------------------
# ЧАСТЬ 2. ДЕТАЛИЗАЦИЯ (ИИ) - ЕДИНЫЙ ДИАЛОГ
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ТЕПЕРЬ Я ЗАДАМ НЕСКОЛЬКО УТОЧНЯЮЩИХ ВОПРОСОВ")
print("=" * 60)

# Формируем промпт с использованием points
if complaints:
    points = "\n".join([f"   • {жалоба}" for жалоба in complaints])
    
    system_prompt = f"""Ты — эмпатичный медицинский ассистент Людмила. 
Пациент {name} подтвердил, что его беспокоят следующие жалобы:

{points}

ТВОЯ ЗАДАЧА:
Проведи детализированный расспрос по КАЖДОЙ жалобе из этого списка, как это делает опытный врач.

ПРАВИЛА ВЕДЕНИЯ БЕСЕДЫ:
1. Начинай с первой жалобы и последовательно опрашивай по всем пунктам
2. По каждой жалобе выясни:
   - Когда началось?
   - Как часто проявляется?
   - Что провоцирует/усиливает?
   - Что облегчает?
   - Пытался ли лечиться самостоятельно?
3. Только когда получишь ответы на эти вопросы по текущей жалобе, переходи к следующей
4. Если пациент упоминает новую проблему, не указанную в списке, вежливо вернись к опросу по списку

⚠️ КРИТИЧЕСКИ ВАЖНО: НЕ завершай беседу, пока не опросишь по ВСЕМ жалобам!

🚫 УСЛОВИЯ ДОСРОЧНОГО ЗАВЕРШЕНИЯ (ТОЛЬКО ПРИ ЭТИХ ТОЧНЫХ ФРАЗАХ):
Пациент должен явно сказать одну из фраз:
- "я устал от беседы"
- "закончим"
- "хватит"
- "передайте врачу"
- "в другой раз"
- "давай закончим"
- "завершаем"

✅ ЕСЛИ ПАЦИЕНТ НЕ ГОВОРИТ ЭТИХ ФРАЗ - ПРОДОЛЖАЙ ОПРОС! Даже если он говорит "я устал" (без "от беседы") - это может быть жалоба на усталость, а не просьба закончить.

В конце беседы (после опроса всех жалоб или при досрочном завершении) скажи: "Спасибо, я собрала всю необходимую информацию и передаю её врачу"."""

    # Правильно инициализируем messages с чередованием ролей
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Здравствуйте! У меня есть жалобы. Начнём опрос."},
        {"role": "assistant", "content": f"Давайте подробнее разберем ваши жалобы. Расскажите о первой: {complaints[0]}. Когда это началось?"}
    ]
    
    print(f"\n🤖 Людмила: Давайте подробнее разберем ваши жалобы. Расскажите о первой: {complaints[0]}. Когда это началось?")
    
    # Собираем полную стенограмму диалога
    full_dialogue = [f"🤖 Людмила: Давайте подробнее разберем ваши жалобы. Расскажите о первой: {complaints[0]}. Когда это началось?"]
else:
    messages = [
        {"role": "system", "content": f"Ты — эмпатичный медицинский ассистент Людмила. Пациент {name} сообщил, что его ничего не беспокоит. Поблагодари его за уделенное время и скажи, что если появятся жалобы, он всегда может обратиться. Не задавай дополнительных вопросов."},
        {"role": "user", "content": "У меня нет жалоб."},
        {"role": "assistant", "content": f"Рада была познакомиться, {name}! Если появятся вопросы или жалобы, обращайтесь. Всего доброго!"}
    ]
    print(f"\n🤖 Людмила: Рада была познакомиться, {name}! Если появятся вопросы или жалобы, обращайтесь. Всего доброго!")
    full_dialogue = [f"🤖 Людмила: Рада была познакомиться, {name}! Если появятся вопросы или жалобы, обращайтесь. Всего доброго!"]

# Основной цикл диалога
patient_wants_to_stop = False

while complaints and not patient_wants_to_stop:
    user_input = input("👤 Вы: ")
    full_dialogue.append(f"👤 Вы: {user_input}")
    
    # РАСШИРЕННАЯ ПРОВЕРКА НА ЗАВЕРШЕНИЕ (только явные фразы)
    exit_phrases = [
        "я устал от беседы",
        "закончим", 
        "хватит опроса",
        "передайте врачу",
        "в другой раз", 
        "давай закончим",
        "завершаем",
        "хватит вопросов"
    ]
    
    # Проверяем точное совпадение фраз или очень близкое
    should_exit = False
    user_lower = user_input.lower()
    for phrase in exit_phrases:
        if phrase in user_lower or user_lower in phrase:
            should_exit = True
            break
    
    if should_exit:
        patient_wants_to_stop = True
        # Добавляем сообщение пользователя
        messages.append({"role": "user", "content": user_input})
        
        response = client.chat.completions.create(
            model="local-model",
            messages=messages,
            temperature=0.7
        )
        
        final_response = response.choices[0].message.content
        messages.append({"role": "assistant", "content": final_response})
        full_dialogue.append(f"🤖 Людмила: {final_response}")
        print(f"\n🤖 Людмила: {final_response}")
        break
    
    # Добавляем ответ пользователя
    messages.append({"role": "user", "content": user_input})
    
    response = client.chat.completions.create(
        model="local-model",
        messages=messages,
        temperature=0.7
    )
    
    next_response = response.choices[0].message.content
    # Добавляем ответ ассистента
    messages.append({"role": "assistant", "content": next_response})
    full_dialogue.append(f"🤖 Людмила: {next_response}")
    print(f"\n🤖 Людмила: {next_response}")
    
    # Проверяем, не завершила ли модель беседу сама (после опроса всех жалоб)
    if "передаю её врачу" in next_response.lower() and "собрала всю необходимую информацию" in next_response.lower():
        break

# ------------------------------------------------------------
# ЧАСТЬ 3. ФОРМИРОВАНИЕ РЕЗЮМЕ ДЛЯ ВРАЧА
# ------------------------------------------------------------
if complaints and full_dialogue:
    # Создаем промпт для формирования резюме
    summary_prompt = f"""Ты — медицинский ассистент Людмила. На основе стенограммы беседы с пациентом {name} составь КРАТКОЕ СТРУКТУРИРОВАННОЕ РЕЗЮМЕ для врача.

СТЕНОГРАММА БЕСЕДЫ:
{chr(10).join(full_dialogue)}

ТВОЯ ЗАДАЧА:
Составь резюме в следующем формате:

ЖАЛОБА 1: [краткое название]
   • Начало: [когда возникла]
   • Характер: [как проявляется, частота]
   • Провоцирующие факторы: [что усиливает]
   • Облегчающие факторы: [что помогает]
   • Попытки лечения: [чем лечился, эффект]

ЖАЛОБА 2: [краткое название]
   • Начало: ...
   ... и так далее для всех жалоб

ДОПОЛНИТЕЛЬНО:
   • Образ жизни/контекст: [важные детали о работе, стрессе, привычках]
   • Особые замечания: [что важно учесть врачу]

ПРАВИЛА:
- Только факты из беседы, без домысливания
- Максимально сжато, но информативно
- Если информации по какому-то пункту нет - пропусти его
- Не используй маркдаун, только простой текст"""

    summary_messages = [
        {"role": "system", "content": "Ты - медицинский ассистент, составляешь краткое резюме беседы для врача."},
        {"role": "user", "content": summary_prompt}
    ]
    
    try:
        summary_response = client.chat.completions.create(
            model="local-model",
            messages=summary_messages,
            temperature=0.3  # Низкая температура для более фактологического ответа
        )
        medical_summary = summary_response.choices[0].message.content
    except Exception as e:
        medical_summary = "Не удалось сформировать резюме автоматически."

# ------------------------------------------------------------
# ЗАВЕРШЕНИЕ (если модель сама не попрощалась)
# ------------------------------------------------------------
if complaints and not patient_wants_to_stop and not any("передаю её врачу" in msg.lower() for msg in full_dialogue[-3:]):
    print("\n" + "=" * 60)
    print(f"🤖 Людмила: Спасибо за беседу, {name}! Я передаю всю информацию вашему врачу.")
    print("🤖 Людмила: До встречи на приёме!")
    print("=" * 60)

# ------------------------------------------------------------
# СОХРАНЕНИЕ СПРАВКИ ДЛЯ ВРАЧА
# ------------------------------------------------------------
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")
spravka = f"""
СПРАВКА ПО РЕЗУЛЬТАТАМ БЕСЕДЫ
================================
Пациент: {name}
Дата: {timestamp}

СОБРАННЫЕ ЖАЛОБЫ:
{'-' * 40}
"""
for i, complaint in enumerate(complaints, 1):
    spravka += f"{i}. {complaint}\n"

if corrections:
    spravka += f"\nКОРРЕКТИРОВКИ ПАЦИЕНТА:\n"
    for corr in corrections:
        spravka += f"   • {corr}\n"

# Добавляем резюме для врача
if complaints and full_dialogue:
    spravka += f"""

📋 РЕЗЮМЕ ДЛЯ ВРАЧА (СФОРМИРОВАНО ИИ)
{'-' * 40}
{medical_summary}
"""

spravka += f"""

📜 ПОЛНАЯ СТЕНОГРАММА БЕСЕДЫ:
{'-' * 40}
"""
for line in full_dialogue:
    spravka += line + "\n"

spravka += """
================================
Сформировано ИИ-ассистентом Людмила
Требуется проверка врача на приёме
"""

# Сохраняем в файл
filename = f"spravka_{name}_{datetime.now().strftime('%Y%m%d_%H%M')}.txt"
with open(filename, "w", encoding="utf-8") as f:
    f.write(spravka)

print(f"\n✅ Справка сохранена в файл: {filename}")
print("\n" + "=" * 60)
print("Справка включает:")
print("• Краткое резюме для быстрого ознакомления")
print("• Полную стенограмму для детального изучения")
print("=" * 60)


ЗДРАВСТВУЙТЕ! Я ЛЮДМИЛА, МЕДИЦИНСКИЙ АССИСТЕНТ

Я помогу собрать информацию о вашем самочувствии
перед беседой с врачом. Всё, что вы расскажете,
я передам доктору.
------------------------------------------------------------



👤 Как вас зовут?  Иван



🤖 Людмила: Очень приятно, Иван!

🤖 Людмила: Расскажите, пожалуйста о том, что Вас беспокоит?


👤 Вы:  У меня проблемы с потенцией



🤖 Людмила: Может что-то ещё?


...


🤖 Людмила: Спасибо за подробное описание. Теперь, чтобы лучше понять ситуацию и определить дальнейшие шаги, давайте еще раз уточним несколько моментов:

*   **Проблемы с потенцией:** Как часто вы испытываете проблемы с потенцией? Это происходит всегда или только в определенных ситуациях (например, при определенной нагрузке)?
*   **Чувство вины:** Что именно вызывает чувство вины из-за проблем с потенцией? Чувствуете ли вы вину за то, что член быстро обмякает? Или за то, что не можете достичь оргазма? За все вместе?
*   **Беспокойство перед близостью:** Что конкретно вызывает беспокойство перед близостью? Страх неудачи? Страх разочаровать партнера? Неуверенность в себе?

Пожалуйста, ответьте на эти вопросы максимально подробно. Ваша честность поможет мне лучше понять вашу ситуацию и подобрать наиболее подходящий план дальнейшего обследования и лечения.



👤 Вы:  Проблемы чаще возникают на фоне подавленного настроения. Тревогу вызывает страх неудачи, а чувство вины приходит за провал. 



🤖 Людмила: Спасибо за уточнение. Итак, основные жалобы:

1.  **Проблемы с потенцией:** Чаще возникают при подавленном настроении. Связаны с опасением неудачи и чувством вины за провал. Частота возникновения связана с уровнем подавленности.
2.  **Чувство вины:** Вызывается страхом неудачи и ощущением провала, что приводит к чувству вины.
3.  **Беспокойство перед близостью:** Связано со страхом неудачи и беспокойством за партнера.

Теперь я передаю эту информацию врачу для дальнейшего обсуждения и планирования лечения. Спасибо за вашу открытость!



👤 Вы:  Спасибо



🤖 Людмила: Спасибо, я собрала всю необходимую информацию и передаю её врачу.


✅ Справка сохранена в файл: spravka_Иван_20260222_1134.txt

Справка включает:
• Краткое резюме для быстрого ознакомления
• Полную стенограмму для детального изучения
